In [ ]:
import os
from glob import glob
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
from numpy.core.defchararray import find
from pandas.api.types import CategoricalDtype
from scipy import stats
from sklearn import linear_model, preprocessing
from sklearn.metrics import (RocCurveDisplay, auc, confusion_matrix,
                             mean_squared_error, roc_curve)
from sklearn.model_selection import KFold, RandomizedSearchCV, train_test_split
from sklearn.svm import SVC
from sklearn.utils.fixes import loguniform
from torch.utils.data import DataLoader, TensorDataset

from label_flip_revised.simple_nn_model import SimpleModel
from label_flip_revised.torch_utils import evaluate, train_model
from label_flip_revised.utils import create_dir, open_csv


In [ ]:
PATH_ROOT = Path(os.getcwd()).absolute().parent
print(PATH_ROOT)

In [ ]:
path_cm = sorted(glob(os.path.join(PATH_ROOT, 'results', 'synth_rand', '*.csv')))
print(len(path_cm))
print(path_cm[:5])

In [ ]:
df_cm = pd.DataFrame()
for p in path_cm:
    _df = pd.read_csv(p)
    df_cm = pd.concat([df_cm, _df], ignore_index=True)

In [ ]:
# Add rates
rates = [float(Path(d).stem.split('_')[-1]) for d in df_cm['Data'].to_list()]
df_cm['Rate'] = rates
df_cm = df_cm[df_cm['Rate'] <= 0.41]
print(df_cm.shape)

In [ ]:
# Add filepath
df_cm['Filepath'] = df_cm['Data'].apply(lambda x: os.path.join('data', 'synth', 'rand', x))
df_cm['Data'] = df_cm['Data'].apply(lambda x: x[:-len('_random_0.05.csv')])

In [ ]:
path_clean = sorted(glob(os.path.join(PATH_ROOT, 'results', 'synth_svm', '*clean*')))
df_cm_clean = pd.DataFrame()
for p in path_clean:
    _df = pd.read_csv(p)
    df_cm_clean = pd.concat([df_cm_clean, _df], ignore_index=True)

df_cm_clean['Rate'] = 0
df_cm_clean['Filepath'] = df_cm_clean['Data'].apply(lambda x: os.path.join('data', 'synth', 'train', '{}_clean_train.csv'.format(x.split('.')[0])))
df_cm_clean['Data'] = df_cm_clean['Data'].apply(lambda x: x.split('.')[0])

df_cm = pd.concat([df_cm_clean, df_cm], ignore_index=True)

In [ ]:
# Remove NA
print('# of columns before removing NA:', len(df_cm.columns) - 1)  # Name does not count
cols_not_na = df_cm.columns[df_cm.notna().any()].tolist()
df_cm = df_cm[cols_not_na]
print('# of columns after removing NA:', len(df_cm.columns) - 1)  # Name does not count
print(cols_not_na)

In [ ]:
df_cm = df_cm.sort_values(by=['Data', 'Rate'], ignore_index=True)
df_cm['Testpath'] = df_cm['Data'].apply(lambda x: os.path.join('data', 'synth', 'test', f'{x}_clean_test.csv'))

In [ ]:
df_cm['Train'] = 0.
df_cm['Test'] = 0.

In [ ]:
COL_NAMES = ['Data', 'F1', 'F1 SD', 'F1v', 'F2', 'F3', 'F4', 'N1', 
    'N2', 'N2 SD', 'N3 ', 'N3 SD', 'N4', 'N4 SD', 'T1', 'T1 SD', 'LSC', 
    'L1', 'L2', 'L3', 'T2', 'T3', 'T4', 'C1', 'C2', 'Density', 'ClsCoef', 
    'Hubs', 'HubsSD', 'Rate', 'Filepath', 'Testpath', 'Train', 'Test']
new_names_map = {df_cm.columns[i]:COL_NAMES[i] for i in range(len(COL_NAMES))}
df_cm = df_cm.rename(new_names_map, axis=1)

In [ ]:
# Load clean score
def update_score(df_score, rate, df_cm):
    for i in range(df_score.shape[0]):
        data = df_score.loc[i, 'data']
        train = df_score.loc[i, 'train']
        test = df_score.loc[i, 'test']
        idx = df_cm[(df_cm['Data'] == data) & (df_cm['Rate'] == rate)].index
        df_cm.loc[idx, 'Train'] = train
        df_cm.loc[idx, 'Test'] = test    

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    print('Running on CPU!')

HIDDEN_LAYER = 128
LR = 0.001  # Learning rate.
MAX_EPOCHS = 400  # Number of iteration for training.
BATCH_SIZE = 128

In [ ]:
def get_dataloader(X, y, batch_size=BATCH_SIZE, shuffle=True):
    dataset = TensorDataset(torch.from_numpy(X).type(torch.float32),
                                  torch.from_numpy(y).type(torch.int64))
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader

In [ ]:
# Load clean score
df_score_clean = pd.read_csv(os.path.join(PATH_ROOT, 'results', 'synth_clean_score.csv'))
update_score(df_score_clean, 0, df_cm)

In [ ]:
path_output = os.path.join(PATH_ROOT, 'results')
path_model = os.path.join(path_output, 'torch')
create_dir(path_model)

In [ ]:
df_cm.head()

In [ ]:
RATES = [f'{r:.2f}' for r in np.arange(0.05, 0.41, 0.05)]
print(RATES)
postfix_torch = '_SimpleNN.torch'

for rate in RATES:
    print(f'Current poison rate: {rate}...')
    path_score = os.path.join(PATH_ROOT, 'results', f'synth_rand_{rate}_score.csv')
    if os.path.exists(path_score):
        df_score = pd.read_csv(path_score)
        update_score(df_score, rate, df_cm)
    else:
        acc_train = []
        acc_test = []
        datanames = []

        for i in df_cm[df_cm['Rate'] == float(rate)].index:
            dataname = df_cm.iloc[i]['Data']
            datanames.append(dataname)

            path_train = os.path.join(PATH_ROOT, df_cm.iloc[i]['Filepath'])
            path_test = os.path.join(PATH_ROOT, df_cm.iloc[i]['Testpath'])

            X_po, y_po, _ = open_csv(path_train)
            X_test, y_test, _ = open_csv(path_test)
            n_features = X_po.shape[1]

            dataloader_poison = get_dataloader(X_po, y_po, shuffle=True)
            dataloader_test = get_dataloader(X_test, y_test, shuffle=False)

            model = SimpleModel(n_features, hidden_dim=HIDDEN_LAYER, output_dim=2).to(device)
            optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=0.8)
            loss_fn = nn.CrossEntropyLoss()

            dataname_poison = '{}_nn_rand_{}{}'.format(dataname, rate, postfix_torch)
            _path_model = os.path.join(path_model, dataname_poison)
            if os.path.exists(_path_model):
                model.load_state_dict(torch.load(_path_model, map_location=device))
            else:
                train_model(model, dataloader_poison, optimizer, loss_fn, device, MAX_EPOCHS)
                torch.save(model.state_dict(), _path_model)

            acc_po, _ = evaluate(dataloader_poison, model, loss_fn, device)
            acc_te, _ = evaluate(dataloader_test, model, loss_fn, device)
            acc_train.append(acc_po)
            acc_test.append(acc_te)

            if i % 10 == 0:
                print(f'[{dataname}] Acc train: {acc_po*100:.2f} test: {acc_te*100:.2f}')
        results = {
            'data': datanames,
            'train': acc_train,
            'test': acc_test,
        }
        df_score = pd.DataFrame(results)
        df_score.to_csv(path_score, index=False)
        update_score(df_score, float(rate), df_cm)


In [ ]:
df_cm[df_cm['Train'] == 0]

In [ ]:
df_cm.to_csv(os.path.join(PATH_ROOT, 'results', 'synth_cmeasures_nn_rand.csv'), index=False)